In [1]:
!pip install pretrainedmodels
!pip install torchsummary

     |████████████████████████████████| 61kB 2.9MB/s eta 0:00:011
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=7b6c4528fe545634fc2de98077ec6b8be780c0463b0332738f5831c1efe3f470
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

import torch.nn.functional as F
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import pretrainedmodels
import os
from torch.utils.data import Dataset
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import time
import numpy as np
import pandas as pd
import os
import datetime as dt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models

from torchsummary import summary


!pip install ipdb
import ipdb


%matplotlib inline

PyTorch Version:  1.5.0+cu101
Torchvision Version:  0.6.0+cu101


In [2]:
data_path = "./data/train/train"
test_path = "./data/test/test"
extraimage_path = "./data/extraimages/extraimages"

path_to_drive = "../../../../content/gdrive/My\ Drive/data/cassava_disease_classification/"

In [3]:
print('Train set:')
class_distrbution = {}
for cls in os.listdir(data_path):
    print('{}:{}'.format(cls, len(os.listdir(os.path.join(data_path, cls)))))
    class_distrbution[cls] =  len(os.listdir(os.path.join(data_path, cls)))
im = Image.open(data_path+'/cgm/train-cgm-738.jpg')
print(im.size)
class_distrbution

Train set:
cbb:466
healthy:316
cbsd:1443
cmd:2658
cgm:773
(500, 500)


{'cbb': 466, 'cbsd': 1443, 'cgm': 773, 'cmd': 2658, 'healthy': 316}

In [4]:
# Transformations for both the training and testing data
mean=[0.4543, 0.5137, 0.3240]
std=[0.1949, 0.1977, 0.1661]


s=1
color_jitter = torchvision.transforms.ColorJitter(
            0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s
        )
        
train_2_transforms  =  torchvision.transforms.Compose([
                            torchvision.transforms.RandomResizedCrop(448),
                            torchvision.transforms.RandomHorizontalFlip(),  # with 0.5 probability
                            torchvision.transforms.RandomApply([color_jitter], p=0.8),
                            torchvision.transforms.RandomGrayscale(p=0.2),
                            torchvision.transforms.ToTensor()])
        

# data augumentaion: RandomCrop, VFlip, HFilp, RandomRotate

train_transforms = transforms.Compose([transforms.RandomResizedCrop(448),
                                       transforms.RandomResizedCrop(448), #448, 299, 224, 331
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomRotation(30),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=mean,std=std)])

test_transforms = transforms.Compose([ transforms.Resize(448),
                                       transforms.CenterCrop(448),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=mean,std=std)])

# normalize = transforms.Normalize(mean=mean, std=std)

In [5]:
class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        class_names = {}
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])

                name = str(i)+'-'+className
                if name not in class_names:
                    class_names[name] = 1
                else:
                    class_names[name] += 1
        self.file_list = files
#         print(class_names)
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
        
        return im.view(3, 448, 448), classCategory
#         return im.view(3, 224, 224), classCategory
# #         return im.view(3, 299, 299), classCategory
#         return im.view(3, 331, 331), classCategory   # NASNetLarge 331x331

In [6]:
# class_names

In [7]:
train_data = CassavaDataset(data_path, transform=train_transforms)

test_data = CassavaDataset(test_path, transform=test_transforms)

extraimage_data = CassavaDataset(extraimage_path, transform=train_transforms) #maybe need an other trasforms, I had to change the dataset structure :)

In [8]:
from torch.utils.data.sampler import SubsetRandomSampler

validation_split = 0.2
shuffle_dataset = True
random_seed= 42 #42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

In [9]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

batch_size = 16 #16

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                             sampler=valid_sampler)

# unlabeled_loader = torch.utils.data.DataLoader(extraimage_data, batch_size=batch_size) # to make batch_size work, I had to moove all the unlabeled data in a 0 folder

# test_loader = torch.utils.data.DataLoader(test_data, batch_size=1) # make batch = 1 here 

In [10]:
# # Creating PT data samplers and loaders:
# # train_sampler = SubsetRandomSampler(train_indices)
# # valid_sampler = SubsetRandomSampler(val_indices)
# from torch.utils.data.sampler import SubsetRandomSampler,WeightedRandomSampler
# #-----------------------------------------------------------------------------z

# validation_split = 0.2
# shuffle_dataset = True
# random_seed= 42

# # ipdb.set_trace()
# # Creating data indices for training and validation splits:
# dataset_size = len(train_data)
# indices = list(range(dataset_size))
# split = int(np.floor(validation_split * dataset_size))

# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)

# train_indices, val_indices = indices[split:], indices[:split]


# # get target

# targets=[]
# #data=[]
# file_list =train_data.file_list
# np.random.shuffle(file_list)
# for i in file_list:
#     targets.append(i[0])
#    # data.append(i[2])
    
# target_train=targets[split:]
# # train_set=data[split:]
# # print(len(train_set))

# target_test=targets[:split]
# #test_set=data[:split]



    
# # for i in test_data.file_list:
# #     test_targets.append(i[0])
    
# # target = torch.cat((torch.zeros(int(len(train_data) * 0.99), dtype=torch.long),
# #                     torch.ones(int(len(train_data) * 0.01), dtype=torch.long)))


# #count classes
# class_count = np.unique(target_train ,return_counts=True)[1]
# #print(class_count)
# class_count_test= np.unique(target_test, return_counts=True)[1]


# #--------------------------------------------------------------------
# # get weights train

# weight_train = 1. / class_count
# #print(targets)
# samples_weight_train = weight_train[target_train]
# #print(samples_weight_train )

# samples_weight_train = torch.from_numpy(samples_weight_train)
# #print(samples_weight_train)
# sampler_train = WeightedRandomSampler(samples_weight_train, len(samples_weight_train))

# #### valid



# weight_test = 1. / class_count_test
# #print(targets)
# samples_weight_test = weight_test[target_test]
# #print(samples_weight_train )

# samples_weight_test = torch.from_numpy(samples_weight_test)
# #print(samples_weight_train)
# sampler_valid = WeightedRandomSampler(samples_weight_test, len(samples_weight_test))


# train_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
#                                              sampler=sampler_train)


# valid_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
#                                              sampler=sampler_valid)
# # valid_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
# #                                              sampler=valid_sampler)

# #test_loader = torch.utils.data.DataLoader(test_data, batch_size=1)

In [11]:
# train_loader

In [12]:
# for data,label in valid_loader:
#     print(label)
#     break
# #     

In [13]:
inv_normalize = transforms.Normalize(
   mean= [-m/s for m, s in zip(mean, std)],
   std=[1/s for s in std]
)

def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = inv_normalize(img)# / 2 + 0.5     # unnormalize
    npimg = img.cpu().numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.figure(figsize=(20,20))
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [14]:
# dataiter = iter(unlabeled_loader)
# images, labels = dataiter.next()

# print(labels)
# img_grid = torchvision.utils.make_grid(images)

# matplotlib_imshow(img_grid, one_channel=False)

In [15]:
class Additional(nn.Module):
    def __init__(self, modelA,in_features,nb_classes=5, freeze = False):
        super(Additional, self).__init__()
        self.modelA = modelA
        # Remove last linear layer
#         self.modelA.fc = nn.Identity() # for resnet
        self.modelA.last_linear = nn.Identity() #for re_renext
#         self.modelA.classifier = nn.Identity()    # densenet201
        for p in self.modelA.parameters():
            if freeze:
                p.requires_grad = False
            else :
                p.requires_grad = True
        
        # Create new classifier
        self.fc_1 = nn.Linear(in_features, 256)
        self.fc_2 = nn.Linear(256, 512)
        self.fc_out = nn.Linear( 512, nb_classes)
        
        #Dropout
        self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, x):
        #model
        x = self.modelA(x.clone())  
        x = x.view(x.size(0), -1)
        
        #FC
        x  = self.dropout(self.fc_1(F.relu(x)))
        x = self.dropout(self.fc_2(F.relu(x)))
        x = self.fc_out(F.relu(x))
        
        return x

In [16]:
# this function is used during training process, to calculation the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [17]:
total_loss_train, total_acc_train = [],[]
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0)
        # print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [18]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [19]:
# import ipdb
# {'0-cbsd': 1443, '1-cgm': 773, '2-cbb': 466, '3-healthy': 316, '4-cmd': 2658}

In [20]:
def test(test_loader, model):
    model.eval()
    pred = []
    _class_labels = np.array(['cbsd','cgm','cbb','healthy','cmd'])
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            images, _ = data
            images = Variable(images).to(device)
    
            outputs = model(images)
    
            prediction = outputs.data.cpu().numpy().argmax()
            
            _predicted_class_labels = _class_labels[prediction]
            
            pred.append(_predicted_class_labels)

    return pred

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'se_resnext101_32x4d' # se_resnext101_32x4d, resnext101_64x4d, nasnetlarge
# resnet_model = torch.hub.load('pytorch/vision:v0.5.0', model_name, pretrained=True)
resnet_model = pretrainedmodels.se_resnext101_32x4d(num_classes=1000, pretrained="imagenet")# todo : how to pretrained=False ?

# # num_fits = resnet_model.fc.in_features
num_fits = 131072# resnet_model.last_linear.in_features # se_resnext101_32x4d
# # num_fits = resnet_model.classifier.in_features # densenet201
# num_fits


model = Additional(resnet_model, num_fits, freeze = False)
model = model.to(device)
# # model

# #--------------------------------------------

In [22]:
model.load_state_dict(torch.load('se_resnext101_32x4dall_data_0.89.ckpt'))

<All keys matched successfully>

In [23]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# train_loader.dataset.classes,
# class_weights = [class_distrbution[i] for i in train_loader.dataset.classes]
# class_weights_normalized = [max(class_weights)/i for i in class_weights]

# class_weights_normalized,torch.Tensor(class_weights_normalized)

# x = torch.Tensor(class_weights_normalized)
# x = x.to(device)
# x = x
# x,class_distrbution

In [24]:
# 76 - 74 (non weight)

In [25]:
lr = 2e-4 # 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


epoch_num = 100
best_val_acc = 0.88
total_loss_val, total_acc_val = [],[]
for epoch in range(1, epoch_num+1):
    loss_val, acc_val = train(train_loader, model, criterion, optimizer, epoch)
#     loss_val, acc_val = validate(valid_loader, model, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        torch.save(model.state_dict(), model_name+'all_data_'+str(best_val_acc)[:4]+'.ckpt')
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

[epoch 1], [iter 100 / 283], [train loss 0.31900], [train acc 0.88875]
[epoch 1], [iter 200 / 283], [train loss 0.30732], [train acc 0.88750]
*****************************************************
best record: [epoch 1], [val loss 0.31581], [val acc 0.88621]
*****************************************************
[epoch 2], [iter 100 / 283], [train loss 0.32153], [train acc 0.88625]
[epoch 2], [iter 200 / 283], [train loss 0.32538], [train acc 0.88594]
[epoch 3], [iter 100 / 283], [train loss 0.32519], [train acc 0.88187]
[epoch 3], [iter 200 / 283], [train loss 0.31756], [train acc 0.88438]
*****************************************************
best record: [epoch 3], [val loss 0.31830], [val acc 0.88704]
*****************************************************
[epoch 4], [iter 100 / 283], [train loss 0.28207], [train acc 0.90250]
[epoch 4], [iter 200 / 283], [train loss 0.28793], [train acc 0.89625]
*****************************************************
best record: [epoch 4], [val loss 0.29

In [27]:
loss_val, acc_val = validate(valid_loader, model, criterion, optimizer, epoch)

------------------------------------------------------------
[epoch 100], [val loss 0.12220], [val acc 0.95551]
------------------------------------------------------------


In [21]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model_name = 'se_resnext101_32x4d' # se_resnext101_32x4d, resnext101_64x4d, nasnetlarge
# # resnet_model = torch.hub.load('pytorch/vision:v0.5.0', model_name, pretrained=True)
# resnet_model = pretrainedmodels.se_resnext101_32x4d(num_classes=1000, pretrained="imagenet")# todo : how to pretrained=False ?

# resnet_model.last_linear = nn.Identity()


# resnet_model.to(device)
# summary(resnet_model, (3, 448, 448))

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'se_resnext101_32x4d' # se_resnext101_32x4d, resnext101_64x4d, nasnetlarge
# resnet_model = torch.hub.load('pytorch/vision:v0.5.0', model_name, pretrained=True)
resnet_model = pretrainedmodels.se_resnext101_32x4d(num_classes=1000, pretrained="imagenet")# todo : how to pretrained=False ?

# resnet_model = torchvision.models.resnet50(pretrained=True)
#---------------------------------------------

# num_fits = resnet_model.fc.in_features
num_fits = resnet_model.last_linear.in_features # se_resnext101_32x4d #131072
# num_fits = resnet_model.classifier.in_features # densenet201
num_fits


model = Additional(resnet_model, num_fits, freeze = False)
model = model.to(device)
# model

#---------------------------------------------

lr = 2e-4 # 0.001
# criterion = nn.CrossEntropyLoss(weight = x)
criterion = nn.CrossEntropyLoss()

# criterion2 = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=lr)


epoch_num = 20
best_val_acc = 0.89
total_loss_val, total_acc_val = [],[]
for epoch in range(1, epoch_num+1):
    loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch)
    loss_val, acc_val = validate(valid_loader, model, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        torch.save(model.state_dict(), model_name+'non_freeze_'+str(best_val_acc)[:4]+'.ckpt')
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

[epoch 1], [iter 100 / 283], [train loss 1.11986], [train acc 0.59375]
[epoch 1], [iter 200 / 283], [train loss 0.99001], [train acc 0.65063]
------------------------------------------------------------
[epoch 1], [val loss 0.70427], [val acc 0.74840]
------------------------------------------------------------
[epoch 2], [iter 100 / 283], [train loss 0.68194], [train acc 0.76500]
[epoch 2], [iter 200 / 283], [train loss 0.67151], [train acc 0.76781]
------------------------------------------------------------
[epoch 2], [val loss 0.54760], [val acc 0.82706]
------------------------------------------------------------
[epoch 3], [iter 100 / 283], [train loss 0.57196], [train acc 0.80937]
[epoch 3], [iter 200 / 283], [train loss 0.58535], [train acc 0.80750]
------------------------------------------------------------
[epoch 3], [val loss 0.53240], [val acc 0.84075]
------------------------------------------------------------
[epoch 4], [iter 100 / 283], [train loss 0.58169], [train acc

In [22]:
# model

In [48]:
# torch.save(model.state_dict(), model_name+str(best_val_acc)[:4]+'.pth')

In [29]:
# # load better model
# model.load_state_dict(torch.load('se_resnext101_32x4dnon_freeze_0.88.ckpt'))

# loss_val, acc_val = validate(valid_loader, model, criterion, optimizer, 1)
# model.eval()

<All keys matched successfully>

In [26]:
loss_val, acc_val = validate(valid_loader, model, criterion, optimizer, 1)

------------------------------------------------------------
[epoch 1], [val loss 0.41739], [val acc 0.87148]
------------------------------------------------------------


### Test Time Augmentation TTA

In [23]:
!pip install git+https://github.com/qubvel/ttach

import ttach as tta

  Cloning https://github.com/qubvel/ttach to /tmp/pip-req-build-sbeuzxxd
  Running command git clone -q https://github.com/qubvel/ttach /tmp/pip-req-build-sbeuzxxd
  Created wheel for ttach: filename=ttach-0.0.2-cp37-none-any.whl size=9084 sha256=59c445b5709a5b782539247542df2cd29892b9201e848be5b7dffe53f33636b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-o1noqaun/wheels/b3/ed/f4/23b671cfba58ac389bbeb0ac6b178f34ee1840422635edd496
Successfully built ttach


In [24]:
transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 180]),
#         tta.Scale(scales=[1, 2, 4]),
#         tta.Multiply(factors=[0.9, 1, 1.1]),        
    ]
)

tta_model = tta.ClassificationTTAWrapper(model, transforms)

loss_val, acc_val = validate(valid_loader, tta_model, criterion, optimizer, 5)

# # #---------------------------------------------

# lr = 2e-4 # 0.001
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=lr)


# epoch_num = 5
# best_val_acc = 0.88
# total_loss_val, total_acc_val = [],[]

# for epoch in range(1, epoch_num+1):
#     loss_train, acc_train = train(train_loader, tta_model, criterion, optimizer, epoch)
#     loss_val, acc_val = validate(valid_loader, tta_model, criterion, optimizer, epoch)
#     total_loss_val.append(loss_val)
#     total_acc_val.append(acc_val)
#     if acc_val > best_val_acc:
#         best_val_acc = acc_val
#         torch.save(model.state_dict(), model_name+'freeze_'+str(best_val_acc)[:4]+'.ckpt')
#         print('*****************************************************')
#         print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
#         print('*****************************************************')
        
# # tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform())

------------------------------------------------------------
[epoch 5], [val loss 0.36979], [val acc 0.87860]
------------------------------------------------------------


In [25]:
acc_val

0.878601152368758

In [43]:
# tta_model

In [ ]:
loss_val, acc_val = validate(train_loader, tta_model, criterion, optimizer, epoch)

## Pseudo Labelling

In [55]:
T1 = 100
T2 = 700
af = 3

def alpha_weight(epoch):
    if epoch < T1:
        return 0.0
    elif epoch > T2:
        return af
    else:
         return ((epoch-T1) / (T2-T1))*af

In [56]:
best_val_acc

0.87

In [ ]:
total_loss_train, total_acc_train = [],[]
def semi_superv_train(train_loader, model, criterion, optimizer, unlabeled_loader, valid_loader, epoch):
    
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(unlabeled_loader):
            images, _ = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(unlabeled_loader):
        images, labels = data
        N = images.size(0)
        # print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
#         labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [60]:
# Concept from : https://github.com/peimengsui/semi_supervised_mnist

from tqdm.notebook import tqdm

acc_scores = []
unlabel = []
pseudo_label = []

alpha_log = []
test_acc_log = []
test_loss_log = []

best_val_acc = 0.87

def semisup_train(train_loader, model, criterion, optimizer, unlabeled_loader, valid_loader, epoch):
#     optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
#     EPOCHS = 5
    
    # Instead of using current epoch we use a "step" variable to calculate alpha_weight
    # This helps the model converge faster
    step = 100 
    
    model.train()
    # for epoch in tqdm(range(EPOCHS)):
    for epoch in range(epoch):

#         for batch_idx, x_unlabeled in enumerate(unlabeled_loader):
        for i, x_unlabeled in enumerate(unlabeled_loader):
            
            
            # Forward Pass to get the pseudo labels
            x_unlabeled = x_unlabeled[0].to(device)
            
            
            model.eval()
            
            with torch.no_grad():
                output_unlabeled = model(x_unlabeled)
                pseudo_labeled = output_unlabeled.max(1, keepdim=True)[1]
                
                
            model.train()
            
            # Now calculate the unlabeled loss using the pseudo label
            output = model(x_unlabeled)
            
            pseudo_labeled = Variable(pseudo_labeled).to(device)
            output = Variable(output).to(device)
            
            unlabeled_loss = alpha_weight(step) * F.nll_loss(output, pseudo_labeled).tem()   
            
            # Backpropogate
            optimizer.zero_grad()
            unlabeled_loss.backward()
            optimizer.step()
            
            
            # For every 50 batches train one epoch on labeled data 
            if i % 50 == 0:
                
                # Normal training procedure
                for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
                    
                    X_batch = Variable(X_batch).to(device)
                    y_batch = Variable(y_batch).to(device)
                    
                    output = model(X_batch)
                    
                    labeled_loss = F.nll_loss(output, y_batch)

                    optimizer.zero_grad()
                    labeled_loss.backward()
                    optimizer.step()
                
                # Now we increment step by 1
                step += 1
                

        loss_val, acc_va = validate(val_loader, model, criterion, optimizer, epoch) # evaluate(model, test_loader)
        
        print('Epoch: {} : Alpha Weight : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, alpha_weight(step), test_acc, test_loss))
        
        if acc_va > best_val_acc:
            best_val_acc = acc_val
            torch.save(model.state_dict(), model_name+'freeze_'+str(best_val_acc)[:4]+'.ckpt')
            print('*****************************************************')
            print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
            print('*****************************************************')
        
#         """ LOGGING VALUES """
#         alpha_log.append(alpha_weight(step))
#         test_acc_log.append(test_acc/100)
#         test_loss_log.append(test_loss)
#         """ ************** """
        model.train()

In [61]:
semisup_train(train_loader, model, criterion, optimizer, unlabeled_loader, valid_loader, 5)

RuntimeError: multi-target not supported at /opt/conda/conda-bld/pytorch_1579022060824/work/aten/src/THCUNN/generic/ClassNLLCriterion.cu:15

In [35]:
# Concept from : https://github.com/peimengsui/semi_supervised_mnist

from tqdm.notebook import tqdm

acc_scores = []
unlabel = []
pseudo_label = []

alpha_log = []
test_acc_log = []
test_loss_log = []

best_val_acc = 0.87

def semisup_train(model, train_loader, unlabeled_loader, val_loader):
#     optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
    EPOCHS = 5
    
    # Instead of using current epoch we use a "step" variable to calculate alpha_weight
    # This helps the model converge faster
    step = 100 
    
    model.train()
    # for epoch in tqdm(range(EPOCHS)):
    for epoch in range(EPOCHS):

        for batch_idx, x_unlabeled in enumerate(unlabeled_loader):
            
            
            # Forward Pass to get the pseudo labels
            x_unlabeled = x_unlabeled[0].to(device)
            model.eval()
            output_unlabeled = model(x_unlabeled)
            _, pseudo_labeled = torch.max(output_unlabeled, 1)
            model.train()
            
            
            """ ONLY FOR VISUALIZATION"""
            if (batch_idx < 3) and (epoch % 10 == 0):
                unlabel.append(x_unlabeled.cpu())
                pseudo_label.append(pseudo_labeled.cpu())
            """ ********************** """
            
            # Now calculate the unlabeled loss using the pseudo label
            output = model(x_unlabeled)
            unlabeled_loss = alpha_weight(step) * criterion(output, pseudo_labeled)   
            
            # Backpropogate
            optimizer.zero_grad()
            unlabeled_loss.backward()
            optimizer.step()
            
            
            # For every 2 batches train one epoch on labeled data 
            if batch_idx % 2 == 0:
                
                # Normal training procedure
                for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
                    X_batch = X_batch.to(device)
                    y_batch = y_batch.to(device)
                    output = model(X_batch)
                    labeled_loss = criterion(output, y_batch)

                    optimizer.zero_grad()
                    labeled_loss.backward()
                    optimizer.step()
                
                # Now we increment step by 1
                step += 1
                

        loss_val, acc_va = validate(val_loader, model, criterion, optimizer, epoch) # evaluate(model, test_loader)
        
        print('Epoch: {} : Alpha Weight : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, alpha_weight(step), test_acc, test_loss))
        
        if acc_va > best_val_acc:
            best_val_acc = acc_val
            torch.save(model.state_dict(), model_name+'freeze_'+str(best_val_acc)[:4]+'.ckpt')
            print('*****************************************************')
            print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
            print('*****************************************************')
        
        """ LOGGING VALUES """
        alpha_log.append(alpha_weight(step))
        test_acc_log.append(test_acc/100)
        test_loss_log.append(test_loss)
        """ ************** """
        model.train()

In [36]:
semisup_train(model, train_loader, unlabeled_loader, valid_loader)

KeyboardInterrupt: 

In [ ]:
total_loss_train, total_acc_train = [],[]
def train(train_loader, model, criterion, optimizer, unlabeled_loader, valid_loader, epoch):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0)
        # print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [196]:
for _,label in 

## Sumission

In [22]:
train_loader.dataset.classes

['cmd', 'healthy', 'cbsd', 'cbb', 'cgm']

In [23]:
class_names = train_loader.dataset.classes # {0:'cbsd', 1: 'cgm', 2: 'cbb', 3: 'healthy', 4: 'cmd'}

In [24]:
def process_image(image_dir):
    # Process a PIL image for use in a PyTorch model
    # tensor.numpy().transpose(1, 2, 0)
    image = Image.open(image_dir)
    preprocess = transforms.Compose([ transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=mean,std=std)])
    image = preprocess(image)
    # Convert 2D image to 1D vector
    image = np.expand_dims(image, 0)
    image = torch.from_numpy(image)
    inputs = image.to(device)
    return inputs

In [25]:
# Using our model to predict the label
def predict(image, model):
    # Pass the image through our model
    output = model(image)
    # Reverse the log function in our output
    output = torch.exp(output)
    # Get the top predicted class, and the output percentage for
    # that class
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

In [26]:
test_directory = "./data/test/test/0"
predictions, test_image_fileName = [], []
try:
    test_images = listdir(test_directory)
    for images in test_images:
        test_image_fileName.append(images)
        image = process_image(f'{test_directory}/{images}')
        top_prob, top_class = predict(image, model)
        predictions.append(class_names[top_class])
except Exception as e:
    print(e)

In [27]:
print("[INFO] Creating pandas dataframe")
submission_data = {"Category":predictions,"Id":test_image_fileName,}
submission_data_frame = pd.DataFrame(submission_data)

[INFO] Creating pandas dataframe


In [28]:
submission_data_frame.head()

,Category,Id
0,cmd,test-img-1551.jpg
1,cmd,test-img-311.jpg
2,cbsd,test-img-600.jpg
3,cmd,test-img-1092.jpg
4,healthy,test-img-156.jpg


In [97]:
submission_data_frame.head()

,Category,Id
0,cbsd,test-img-2547.jpg
1,cbsd,test-img-1415.jpg
2,cbsd,test-img-2683.jpg
3,cbb,test-img-683.jpg
4,cmd,test-img-3585.jpg


In [29]:
submission_data_frame.to_csv('submission'+model_name+'_91_oversampling.csv', header=True, index=False)